# Validation Pipeline

## Import Dependecies

In [1]:
import random
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

from llama_index.core import Settings
from llama_index.core import StorageContext 
from llama_index.core import load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from tqdm import tqdm

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\lrobert\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
c:\Users\lrobert\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set LLM to None
Settings.llm = None

# Set Hugging Face embedding model for LlamaIndex
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L12-v2")

LLM is explicitly disabled. Using MockLLM.


c:\Users\lrobert\anaconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lrobert\AppData\Local\llama_index\models--sentence-transformers--all-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


## Method(s)

In [24]:
def map_found(founds, answer):
    # Készítsünk egy uniót a két listából, hogy megkapjuk az összes egyedi mondatot
    all_sentences = list(set(founds + answer))

    # Bináris címkézés az átfedésekhez
    y_true = [1 if sentence in founds else 0 for sentence in all_sentences]
    y_pred = [1 if sentence in answer else 0 for sentence in all_sentences]

    # Kiértékelés: accuracy, precision, recall, f1 score
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')

    return accuracy, f1, recall, precision

In [5]:
def get_response(resp):
    return resp.split("\n---------------------\n")[1].split("\n\n")

## Load Data

In [6]:
dir_data = "../data/"

In [7]:
medquad = pd.read_json(f"{dir_data}validations/mqdquad.json", orient="records")
medquad.info()
medquad.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16407 entries, 0 to 16406
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  16407 non-null  object
 1   answer    16407 non-null  object
dtypes: object(2)
memory usage: 256.5+ KB


,question,answer
0,What is (are) keratoderma with woolly hair ?,Keratoderma with woolly hair is a group of rel...
1,How many people are affected by keratoderma wi...,Keratoderma with woolly hair is rare; its prev...
2,What are the genetic changes related to kerato...,"Mutations in the JUP, DSP, DSC2, and KANK2 gen..."
3,Is keratoderma with woolly hair inherited ?,Most cases of keratoderma with woolly hair hav...
4,What are the treatments for keratoderma with w...,These resources address the diagnosis or manag...


In [8]:
len_mq = int(len(medquad)*0.1)
medquad_sp = medquad.sample(n=len_mq, random_state=42).copy()
medquad_sp = medquad_sp.reset_index(drop=True)
medquad_sp.info()
medquad_sp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1640 entries, 0 to 1639
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  1640 non-null   object
 1   answer    1640 non-null   object
dtypes: object(2)
memory usage: 25.8+ KB


,question,answer
0,What are the treatments for dihydropyrimidinas...,These resources address the diagnosis or manag...
1,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
2,What is (are) phenylketonuria ?,Phenylketonuria (commonly known as PKU) is an ...
3,Is Laron syndrome inherited ?,Is Laron syndrome inherited? Most cases of Lar...
4,What are the treatments for globozoospermia ?,These resources address the diagnosis or manag...


## Live

### Sentence Based

In [37]:
storage_context_sb = StorageContext.from_defaults(persist_dir="../data/vectors/sentence_based")
index_sb = load_index_from_storage(storage_context_sb)
query_engine_sb = index_sb.as_query_engine(similarity_top_k=5)

In [39]:
resps = get_response(query_engine_sb.query(medquad["question"].values[0]).response)
map_found(resps, sent_tokenize(medquad["answer"].values[0]))

(0.1111111111111111, 0.2, 0.4, 0.13333333333333333)

In [ ]:
resps = []
for q in tqdm(medquad["question"].values):
    resps.append(get_response(query_engine_sb.query(q).response))

medquad["answer_sb_k1"] = resps

medquad.head()

In [ ]:
accs = [0] * len(medquad)
recalls = [0] * len(medquad)

for i in tqdm(range(0,len(medquad))):
    acc, f1, recall, prec = map_found(
        medquad["answer_sb_k1"].values[i], 
        sent_tokenize(medquad["answer"].values[i]))
    
    #print(acc,f1,recall,prec)
    accs[i] = acc
    # f1s[i] = f1
    recalls[i] = recall
    # precisions[i] = prec

medquad["ACC_sb_k1"] = accs
medquad["RECALL_sb_k1"] = recalls

## Demo

In [ ]:
question  = medquad["question"].values[10]
real_answer  = medquad["answer"].values[10]
fake_answer = medquad["answer"].values[5] + medquad["answer"].values[20]

In [ ]:
fake_answer_sentence = fake_answer.split(".")
fake_answer_sentence = [item.strip() for item in fake_answer_sentence if len(item) > 2]
random.seed(10)
random.shuffle(fake_answer_sentence)
fake_answer_sentence

In [ ]:
real_answer_sentence = real_answer.split(".")
real_answer_sentence = [item.strip() for item in real_answer_sentence if len(item) > 2]
random.seed(10)
random.shuffle(real_answer_sentence)
real_answer_sentence

In [ ]:
demo_found = (real_answer_sentence[:5] + fake_answer_sentence[:3])
random.seed(22)
random.shuffle(demo_found)
demo_found

In [ ]:
acc, f1, recall, precision = map_found(demo_found, real_answer)

print("Accuracy scores: ", acc)
print("f1 scores", f1)
print("Recall scores", recall)
print("Precision scores", precision)
print("Mean Reciprocal Rank (MRR): ", calculate_mrr_from_chunks(demo_found, real_answer))